## Dataloader with words embedding on the fly

In [15]:
import torch.utils.data as data
from itertools import tee
from helpers import *
import pandas as pd
from drop_similar_quotes import *

In [26]:
from glove_embedding import *
GLOVE_EMBEDDING = GLOVE_50D
dimensions = 50
max_sentence_length = 100

In [27]:
GLOVE_EMBEDDING

'data.nosync/glove/glove.6B.50d.txt'

In [28]:
GLOVE_50D

'data.nosync/glove/glove.6B.50d.txt'

In [29]:
def embedding(quote,vocabulary):
    #NICKY's Embedding
    embedded = embed_quote(quote,50,vocabulary)
    return embedded

In [35]:
#This dataloder allows to import Data chunk by chunk, embedding words on the fly 
class DataLoaderSegmentation(data.Dataset):
    #Initalize the dataloader. The JsonReader object will be essential.
    def __init__(self, folder_path,glve,chunksize = 10000):
        super(DataLoaderSegmentation, self).__init__()
        self.datareader = pd.read_json(folder_path, lines=True, compression='bz2', chunksize = chunksize)
        self.itercopy,self.datareader = tee(self.datareader)
        self.chunksize = chunksize
        self.glve = glve
    #Return one chunk of data that has been embedded on the fly
    def next(self):
        try:
            chunk = next(self.datareader) #[['quotation_clean','quoteID','party_label']]
            chunk['quotation_length'] = chunk['quotation_clean'].apply(lambda x: len(x))
            chunk = chunk[chunk['quotation_length'] <=  max_sentence_length]
            chunk = chunk[chunk['quotation_length'] > 0]
        except:
            self.itercopy,self.datareader = tee(self.itercopy)
            chunk = next(self.datareader)
            chunk['quotation_length'] = chunk['quotation_clean'].apply(lambda x: len(x))
            chunk = chunk[chunk['quotation_length'] <=  max_sentence_length]
            chunk = chunk[chunk['quotation_length'] > 0]
        chunk['embedded_quote'] = chunk['quotation_clean'].map(lambda p: embedding(p, self.glve))
        return (chunk)
    
    def __len__(self):
        return len(self.datareader)

In [3]:
file = PREPROCESSED_FOLDER + 'fulldataset_fullypreprocessed.json.bz2'
# glve = load_glove_model(GLOVE_EMBEDDING,dimensions)
# dataloader = DataLoaderSegmentation(folder_path = file,glve = glve)
# chunk = dataloader.next()

In [37]:
chunk.head(50)

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_clean,quotation_length,embedded_quote
1,2015-02-03-074650,The safety and security of our nation is at ri...,jeanne shaheen,2015-02-03 20:02:24,2,Q270316,D,S001181,safety security nation risk hold funding homel...,69,"[[-0.60245, 0.077829, 0.5245, 0.05071, -0.5625..."
2,2015-05-10-027625,It's been my experience that the faith communi...,jeanne shaheen,2015-05-10 22:48:46,1,Q270316,D,S001181,experience faith community willing partner gov...,74,"[[0.35663, 0.61973, -0.71631, 0.20023, 0.2629,..."
3,2015-09-01-113454,we need to bring the parents in so they can se...,jeanne shaheen,2015-09-01 02:12:00,2,Q270316,D,S001181,need bring parents see manufacturing worked sh...,69,"[[0.41495, 0.11293, 0.45591, -0.51796, 0.3896,..."
6,2015-04-15-038299,"If they destroy you, who is there left?",bernie sanders,2015-04-15 17:49:00,1,Q359442,D,S000033,destroy left,12,"[[1.2015, -0.67631, 1.2147, -0.088623, 0.63182..."
7,2015-07-30-060129,It would make everybody in America poorer.,bernie sanders,2015-07-30 01:49:15,13,Q359442,D,S000033,would make everybody america poorer,35,"[[0.7619, -0.29773, 0.51396, -0.13303, 0.24156..."
8,2015-10-14-073757,"labor is the source of all wealth,",bernie sanders,2015-10-14 20:09:23,1,Q359442,D,S000033,labor source wealth,19,"[[-1.0408, -0.33298, -0.029981, -0.40867, -0.3..."
10,2015-04-21-040961,"No, it's America,",bernie sanders,2015-04-21 20:01:11,1,Q359442,D,S000033,america,7,"[[-0.13124, 0.46555, -0.10921, 0.18759, 0.0733..."
11,2015-09-20-040134,People should be elected to office based on th...,bernie sanders,2015-09-20 21:20:20,18,Q359442,D,S000033,people elected office based ideas religion col...,53,"[[0.95281, -0.20608, 0.55618, -0.46323, 0.7335..."
15,2015-12-23-107528,"You know, education, if you make the most of i...",bernie sanders,2015-12-23 21:02:01,1,Q359442,D,S000033,know education make study hard homework make e...,80,"[[0.28328, 0.12887, 0.29861, -0.23238, 0.88263..."
17,2015-08-29-005147,"As our campaign progresses,",bernie sanders,2015-08-29 04:46:14,1,Q359442,D,S000033,campaign progresses,19,"[[0.18605, -0.52757, 0.1689, -0.20537, -0.6967..."


In [19]:
data = pd.read_json(file, lines=True, compression='bz2')
data

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_clean
0,2015-08-31-000271,... a great day for veterans here in Littleton...,jeanne shaheen,2015-08-31 02:10:00,2,Q270316,D,S001181,great day veterans littleton across north coun...
1,2015-02-03-074650,The safety and security of our nation is at ri...,jeanne shaheen,2015-02-03 20:02:24,2,Q270316,D,S001181,safety security nation risk hold funding homel...
2,2015-05-10-027625,It's been my experience that the faith communi...,jeanne shaheen,2015-05-10 22:48:46,1,Q270316,D,S001181,experience faith community willing partner gov...
3,2015-09-01-113454,we need to bring the parents in so they can se...,jeanne shaheen,2015-09-01 02:12:00,2,Q270316,D,S001181,need bring parents see manufacturing worked sh...
4,2015-10-25-000242,"' It is not now, nor has it ever been, the gol...",bernie sanders,2015-10-25 14:12:35,1,Q359442,D,S000033,ever gold standard trade agreements sanders sa...
...,...,...,...,...,...,...,...,...,...
6396366,2020-02-10-100845,We're just worried about making sure we keep t...,sherrie sprenger,2020-02-10 00:00:00,1,Q7495360,R,None,worried making sure keep balance expression ri...
6396367,2020-03-16-079753,We've all embraced strict proper hygiene proce...,robert abrams,2020-03-16 12:00:00,2,Q2156314,D,None,embraced strict proper hygiene procedures hear...
6396368,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,1,Q6501617,D,None,important keep moving forward
6396369,2020-02-20-093793,"When they want a bill, they bring the bill to ...",j.t. wilcox,2020-02-20 22:12:45,1,Q6104393,R,None,want bill bring bill floor


In [20]:
data.sort_values(by = 'speaker', axis=0, ascending=True, inplace=True)
data

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_clean
1182275,2016-05-24-005262,"An injustice has been rectified,",a. donald mceachin,2016-05-24 17:10:24,1,Q4647699,D,M001200,injustice rectified
4636148,2018-08-21-017999,Every day the Trump Administration is making i...,a. donald mceachin,2018-08-21 20:09:57,1,Q4647699,D,M001200,every day trump administration making difficul...
2836170,2017-05-31-097169,She -- and potentially her unborn child -- are...,a. donald mceachin,2017-05-31 18:40:00,1,Q4647699,D,M001200,potentially unborn child suffering someone els...
5442589,2019-04-03-123873,We have to know where we are to figure out whe...,a. donald mceachin,2019-04-03 13:43:54,1,Q4647699,D,M001200,know figure need go instead taking victory lap...
5249268,2019-05-30-031058,I am excited to see these valuable workshops c...,a. donald mceachin,2019-05-30 22:05:14,1,Q4647699,D,M001200,excited see valuable workshops come district
...,...,...,...,...,...,...,...,...,...
4938001,2019-04-16-005921,Any statement that Members and staff have not ...,zoe lofgren,2019-04-16 15:55:08,1,Q218217,D,L000397,statement members staff received anti harassme...
4938002,2019-11-05-067423,Our country urgently needs a legal framework t...,zoe lofgren,2019-11-05 05:00:00,7,Q218217,D,L000397,country urgently needs legal framework protect...
5779278,2019-06-17-074804,The bill would clearly state that DHS could no...,zoe lofgren,2019-06-17 09:48:02,2,Q218217,D,L000397,bill would clearly state dhs could use rely re...
3204416,2017-04-22-029739,"I was told that this rally is political, but n...",zoe lofgren,2017-04-22 21:15:01,1,Q218217,D,L000397,told rally political partisan


In [21]:
data

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_clean
1182275,2016-05-24-005262,"An injustice has been rectified,",a. donald mceachin,2016-05-24 17:10:24,1,Q4647699,D,M001200,injustice rectified
4636148,2018-08-21-017999,Every day the Trump Administration is making i...,a. donald mceachin,2018-08-21 20:09:57,1,Q4647699,D,M001200,every day trump administration making difficul...
2836170,2017-05-31-097169,She -- and potentially her unborn child -- are...,a. donald mceachin,2017-05-31 18:40:00,1,Q4647699,D,M001200,potentially unborn child suffering someone els...
5442589,2019-04-03-123873,We have to know where we are to figure out whe...,a. donald mceachin,2019-04-03 13:43:54,1,Q4647699,D,M001200,know figure need go instead taking victory lap...
5249268,2019-05-30-031058,I am excited to see these valuable workshops c...,a. donald mceachin,2019-05-30 22:05:14,1,Q4647699,D,M001200,excited see valuable workshops come district
...,...,...,...,...,...,...,...,...,...
4938001,2019-04-16-005921,Any statement that Members and staff have not ...,zoe lofgren,2019-04-16 15:55:08,1,Q218217,D,L000397,statement members staff received anti harassme...
4938002,2019-11-05-067423,Our country urgently needs a legal framework t...,zoe lofgren,2019-11-05 05:00:00,7,Q218217,D,L000397,country urgently needs legal framework protect...
5779278,2019-06-17-074804,The bill would clearly state that DHS could no...,zoe lofgren,2019-06-17 09:48:02,2,Q218217,D,L000397,bill would clearly state dhs could use rely re...
3204416,2017-04-22-029739,"I was told that this rally is political, but n...",zoe lofgren,2017-04-22 21:15:01,1,Q218217,D,L000397,told rally political partisan


In [ ]:
datanoduplicate =  remove_duplicates(data,0.7)

In [ ]:
FILEPATH =  PREPROCESSED_FOLDER + 'fulldataset_fullypreprocessed_noduplicates.json.bz2'
datanoduplicate.to_json(path_or_buf=None, orient=None, date_format=None, double_precision=10, force_ascii=True, date_unit='ms', default_handler=None, lines=False, compression='infer', index=True, indent=None, storage_options=None)